In [ ]:
pwd

'/content'

In [ ]:
! pip uninstall flask_ngrok # first uninstalling and reinstalling ngrok resolved some error while deploying

In [ ]:
! pip install flask_ngrok
#! pip install pyngrok

In [ ]:
from __future__ import division, print_function
import sys
import os
import glob
import re
import numpy as np
import cv2

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from keras.models import model_from_json

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
#from gevent.pywsgi import WSGIServer
from flask_ngrok import run_with_ngrok
from flask import send_file

from matplotlib import pyplot as plt


In [ ]:
cd '/content/drive/MyDrive/deployment_final' # changing directory

/content/drive/MyDrive/deployment_final


In [ ]:

# Define a flask app
app = Flask(__name__) #runs on local server
run_with_ngrok(app) #required to run on global server

# Model saved with Keras model.save()
#MODEL_PATH = '/content/drive/MyDrive/my_model.h5'

# Load your trained model

# load json and create model
json_file = open('/content/drive/MyDrive/cnn_model.json', 'r')#model from our main script
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/cnn_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
#score = loaded_model.evaluate(trainImages, trainTargets,  verbose=1)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


#from keras.applications.resnet50 import ResNet50
#model = ResNet50(weights='imagenet')
#model.save('')
#print('Model loaded. Check http://127.0.0.1:5000/')



Loaded model from disk


In [ ]:

def model_predict(img_path, loaded_model):

    img = cv2.imread(img_path)  # convert to array
    img_gray =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    imgTopHat = cv2.morphologyEx(img_gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(img_gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    #Thresholding

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    img_thresh = cv2.adaptiveThreshold(
        img_blurred, 
        maxValue=255.0, 
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        thresholdType=cv2.THRESH_BINARY_INV, 
        blockSize=19, 
        C=9)




    img = cv2.resize(img_thresh, (224, 224))  # resize to normalize data size
    X = np.array(img).reshape(-1, 224, 224,1)#train data--converting to array for input to cnn
    X_norm = np.array(X, dtype="float32") / 255.0 #normalising as pixel ranges from 0 to 255

    #img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    #x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    #x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    #x = preprocess_input(x, mode='caffe')

    preds = loaded_model.predict(X_norm)[0]
    return preds


In [ ]:
UPLOAD_FOLDER = '/content/drive/MyDrive/deployment_final/static/uploads' # directory where uploaded files r going to be saved
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [ ]:
output_path = './static/detections/' # predicted image directory

In [ ]:

@app.route("/") #home page
def index():
  return render_template("index.html")

@app.route("/about") #about page
def about():
  return render_template("about.html")



@app.route('/uploader', methods = ['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']
        # create a secure filename
        filename = secure_filename(f.filename)
        print(filename)
        # save file to /static/uploads
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print(filepath)
        f.save(filepath)
        print('filename is :',secure_filename(f.filename))

        # Make prediction
        preds = model_predict(filepath, loaded_model)

        (startX, startY, endX, endY) = preds
        # load the input image (in OpenCV format), resize it such that it
        # fits on our screen, and grab its dimensions
        image = cv2.imread(filepath)
        #image = cv2.resize(image, (250,250))
        (h, w) = image.shape[:2]
        # scale the predicted bounding box coordinates based on the image
        # dimensions
        startX = int(startX * w)
        startY = int(startY * h)
        endX = int(endX * w)
        endY = int(endY * h)
        # draw the predicted bounding box on the image
        cv2.rectangle(image, (startX, startY), (endX, endY),
          (255, 0, 0), 5)
        # show the output image
        plt.imshow(image)
        result = plt.savefig('/content/drive/MyDrive/deployment_final/static/detections/'+'{}' .format(filename))
        file_path_result = '/content/drive/MyDrive/deployment_final/static/detections/result.jpg'

        return render_template("uploaded.html", display_detection = filename, fname = filename)  
        #return send_file(('/content/result.jpg'), mimetype="image/jpeg", attachment_filename="result.jpg")
    return None


#if __name__ == '__main__':
#app.run()

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d995be671dfd.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Mar/2021 05:50:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:50:58] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:50:59] "GET /favicon.ico HTTP/1.1" 404 -


Cars0.png
/content/drive/MyDrive/deployment_final/static/uploads/Cars0.png
filename is : Cars0.png


...

127.0.0.1 - - [25/Mar/2021 05:56:16] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:56:16] "GET /static/detections/Cars1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:56:16] "GET /static/uploads/Cars1.png HTTP/1.1" 200 -


Cars7.png
/content/drive/MyDrive/deployment_final/static/uploads/Cars7.png
filename is : Cars7.png


127.0.0.1 - - [25/Mar/2021 05:56:33] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:56:33] "GET /static/detections/Cars7.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:56:33] "GET /static/uploads/Cars7.png HTTP/1.1" 200 -


18.jpg
/content/drive/MyDrive/deployment_final/static/uploads/18.jpg
filename is : 18.jpg


127.0.0.1 - - [25/Mar/2021 05:57:11] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:57:11] "GET /static/detections/18.jpg HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 05:57:11] "GET /static/uploads/18.jpg HTTP/1.1" 200 -


In [ ]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18781 sha256=76cd41fa6de29659ba11f06a357817c0568232ffcc538ad1ce832519f7c6fbf1
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [ ]:
! ngrok authtoken 1lrZTxinRo0U9mi6coaB6fLnBjw_64Cp5Skg2bZtc37JVq4w1

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import pyngrok

ModuleNotFoundError: ignored

In [ ]:

from pyngrok import ngrok


In [ ]:
ngrok.kill()

In [ ]:
app.run_server(mode='external')

AttributeError: ignored